#### Installing dependencies

In [ ]:
%pip install peft nltk datasets sentencepiece 'accelerate>=0.26.0' "transformers[torch]" textstat sentence-transformers bert_score

#### Importing libraries

In [5]:
# Use PyTorch with MPS backend
import torch
import random
import nltk
import textstat
import numpy as np
import torch.nn.functional as F

# For Flan T5 Large
from transformers import T5ForConditionalGeneration, T5Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from transformers import pipeline
from collections import Counter
from bert_score import score

# Load the pretrained T5 model (not fine-tuned)
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Download required NLTK data
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")

# Load the dataset for evaluation
from datasets import load_dataset

dataset = load_dataset("gretelai/synthetic_text_to_sql")

#### Initializing pretrained model for Apple MPS

In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"device is {device}")


model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model.to(device)

device is mps


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

#### Defining method for calculating evaluation metrics

In [7]:
# Simple ROUGE implementation without external dependencies
def calculate_rouge_scores(reference, candidate):
    """
    Calculate ROUGE-1, ROUGE-2, ROUGE-L scores
    """
    ref_tokens = set(nltk.word_tokenize(reference.lower()))
    cand_tokens = set(nltk.word_tokenize(candidate.lower()))

    # ROUGE-1
    rouge1 = (
        len(ref_tokens.intersection(cand_tokens)) / len(ref_tokens) if ref_tokens else 0
    )

    # ROUGE-2 (bigrams)
    ref_bigrams = set(
        zip(
            nltk.word_tokenize(reference.lower())[:-1],
            nltk.word_tokenize(reference.lower())[1:],
        )
    )
    cand_bigrams = set(
        zip(
            nltk.word_tokenize(candidate.lower())[:-1],
            nltk.word_tokenize(candidate.lower())[1:],
        )
    )
    rouge2 = (
        len(ref_bigrams.intersection(cand_bigrams)) / len(ref_bigrams)
        if ref_bigrams
        else 0
    )

    # ROUGE-L (longest common subsequence)
    def lcs_length(s1, s2):
        m, n = len(s1), len(s2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i - 1] == s2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
        return dp[m][n]

    ref_words = nltk.word_tokenize(reference.lower())
    cand_words = nltk.word_tokenize(candidate.lower())
    lcs = lcs_length(ref_words, cand_words)
    rouge_l = lcs / len(ref_words) if ref_words else 0

    return rouge1, rouge2, rouge_l


# Calculate METEOR score
def calculate_meteor(reference_sql, generated_sql):
    ref_tokens = [word_tokenize(reference_sql.lower())]
    gen_tokens = word_tokenize(generated_sql.lower())
    score = meteor_score(ref_tokens, gen_tokens)
    return score


# Calculate GLEU score
def calculate_gleu(reference_sql, generated_sql):
    ref_tokens = word_tokenize(reference_sql.lower())
    gen_tokens = word_tokenize(generated_sql.lower())

    # Calculate n-gram overlaps
    def get_ngrams(tokens, n):
        return [tuple(tokens[i : i + n]) for i in range(len(tokens) - n + 1)]

    # GLEU calculation (simplified version)
    ref_1grams = set(get_ngrams(ref_tokens, 1))
    gen_1grams = set(get_ngrams(gen_tokens, 1))

    precision = (
        len(ref_1grams.intersection(gen_1grams)) / len(gen_1grams) if gen_1grams else 0
    )
    recall = (
        len(ref_1grams.intersection(gen_1grams)) / len(ref_1grams) if ref_1grams else 0
    )

    gleu = (
        2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    )
    return gleu


# Calculate Fleash Reading Ease score which measures readability and complexity of the SQL
def calculate_flesch_reading_ease(generated_sql):
    # Convert SQL to more readable format for scoring
    readable_sql = generated_sql.replace("SELECT", "SELECT ").replace("FROM", " FROM ")
    flesch_score = textstat.flesch_reading_ease(readable_sql)
    return flesch_score


# Calculate BERT score
def calculate_bert_score(reference_sql, generated_sql):
    try:
        bert_score = score([generated_sql], [reference_sql], lang="en", verbose=False)
        return bert_score[0].item()  # Return F1 scores
    except Exception as e:
        print(f"Error calculating BERT score for {generated_sql}: {e}")
        return 0


# Calculate CoSIM score
def calculate_cosim_score(reference_sql, generated_sql):
    sentence_transformer_model = SentenceTransformer("all-MiniLM-L6-v2")
    reference_embeddings = sentence_transformer_model.encode(reference_sql)
    generated_embeddings = sentence_transformer_model.encode(generated_sql)
    # Calculate cosine similarity using dot product
    ref_norm = reference_embeddings / np.linalg.norm(reference_embeddings)
    gen_norm = generated_embeddings / np.linalg.norm(generated_embeddings)

    cosim_score = np.dot(ref_norm, gen_norm)
    return cosim_score


def calculate_toxicity_score(generated_sql):
    """
    Calculate toxicity score for a single SQL query
    """
    try:
        # Initialize toxicity analyzer
        toxicity_analyzer = pipeline(
            "text-classification", model="unitary/toxic-bert", return_all_scores=True
        )

        # Analyze toxicity of the SQL query
        results = toxicity_analyzer(generated_sql)
        # Get the maximum toxicity score across all categories
        # Categories: toxic, severe_toxic, obscene, threat, insult, identity_hate
        max_toxicity = max([score["score"] for score in results[0]])

        return max_toxicity

    except Exception as e:
        print(f"Error calculating toxicity score: {e}")
        # Return 0 if toxicity calculation fails
        return 0.0


# Evaluation function - REPLACE YOUR EXISTING FUNCTION WITH THIS
def evaluate_sql_generation(model, tokenizer, test_samples=50, sample_dataset=None):
    """
    Evaluate T5 Large pretrained model on SQL generation
    """
    results = {
        "bleu_scores": [],
        "rouge_1_scores": [],
        "rouge_2_scores": [],
        "rouge_l_scores": [],
        "meteor_scores": [],
        "gleu_scores": [],
        "flesch_reading_ease_scores": [],
        "bert_scores": [],
        "cosim_scores": [],
        "repetition_rates": [],
        "novelty_scores": [],
        "diversity_scores": [],
        "toxicity_scores": [],
    }

    # Get test samples
    if test_samples > 0:
        test_data = dataset["test"].select(
            range(min(test_samples, len(dataset["test"])))
        )
    else:
        test_data = sample_dataset

    generated_sqls = []
    reference_sqls = []
    prompts = []

    for i, example in enumerate(test_data):
        # Prepare input
        input_text = (
            f"Question: {example['sql_prompt']} Context: {example['sql_context']}, IMPORTANT: With or without the context, your answer must always be a SQL statement",
        )

        # Tokenize input
        inputs = tokenizer(
            input_text, return_tensors="pt", max_length=512, truncation=True
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate SQL
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=2,
            )

        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference_sql = example["sql"]

        generated_sqls.append(generated_sql)
        reference_sqls.append(reference_sql)
        prompts.append(example["sql_prompt"])

        # Calculate metrics
        # 1. BLEU Score
        reference_tokens = nltk.word_tokenize(reference_sql.lower())
        generated_tokens = nltk.word_tokenize(generated_sql.lower())
        bleu_score = sentence_bleu(
            [reference_tokens],
            generated_tokens,
            smoothing_function=SmoothingFunction().method1,
        )
        results["bleu_scores"].append(bleu_score)

        # 2. ROUGE Scores
        rouge1, rouge2, rouge_l = calculate_rouge_scores(reference_sql, generated_sql)
        results["rouge_1_scores"].append(rouge1)
        results["rouge_2_scores"].append(rouge2)
        results["rouge_l_scores"].append(rouge_l)

        meteor_score = calculate_meteor(reference_sql, generated_sql)
        results["meteor_scores"].append(meteor_score)

        # 3. GLEU Score
        gleu_score = calculate_gleu(reference_sql, generated_sql)
        results["gleu_scores"].append(gleu_score)

        # 4. Fleash Reading Ease Score
        flesch_reading_ease_score = calculate_flesch_reading_ease(generated_sql)
        results["flesch_reading_ease_scores"].append(flesch_reading_ease_score)

        # 5. BERT Score
        bert_score = calculate_bert_score(reference_sql, generated_sql)
        results["bert_scores"].append(bert_score)

        # 6. CoSIM Score
        cosim_score = calculate_cosim_score(reference_sql, generated_sql)
        results["cosim_scores"].append(cosim_score)

        # 3. Repetition Rate
        # It is the percentage of tokens that are repeated in the generated SQL
        tokens = generated_sql.split()
        if len(tokens) > 0:
            repetition_rate = 1 - len(set(tokens)) / len(tokens)
        else:
            repetition_rate = 0
        results["repetition_rates"].append(repetition_rate)

        if i % 10 == 0:
            print(f"Processed {i+1}/{len(test_data)} samples")

    # Calculate novelty and diversity across all generated SQLs
    # To calculate novelty, we need to count the frequency of each token in the generated SQLs
    # Then we can calculate the average novelty of the generated SQLs

    all_tokens = []
    for sql in generated_sqls:
        all_tokens.extend(nltk.word_tokenize(sql.lower()))

    # 4. Novelty (how different from common patterns)
    token_freq = Counter(all_tokens)
    novelty_scores = []
    for sql in generated_sqls:
        tokens = nltk.word_tokenize(sql.lower())
        avg_novelty = np.mean([1 / (token_freq.get(token, 1) + 1) for token in tokens])
        novelty_scores.append(avg_novelty)
    results["novelty_scores"] = novelty_scores

    # 5. Diversity (unique SQL patterns)
    unique_patterns = len(set(generated_sqls))
    diversity_score = unique_patterns / len(generated_sqls)
    results["diversity_scores"] = [diversity_score] * len(generated_sqls)

    # 6. Toxicity Score
    toxicity_scores = [calculate_toxicity_score(sql) for sql in generated_sqls]
    results["toxicity_scores"] = toxicity_scores

    return results, generated_sqls, reference_sqls, test_data

#### Intializing full fine tuning

In [8]:
#### Full Fine-tuning Setup (No PEFT)

# For full fine-tuning, we don't need PEFT configuration
# The model will be fine-tuned with all parameters

# Enable gradient checkpointing to save memory during full fine-tuning
model.gradient_checkpointing_enable()

# Print model information
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(
    f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%"
)

Total parameters: 783,150,080
Trainable parameters: 783,150,080
Percentage of trainable parameters: 100.00%


#### Preparing train and test datasets

In [9]:
# Preparing train and test datasets
def preprocess_function(examples):
    """
    Preprocess the dataset for Text-to-SQL generation
    """
    # Format input as instruction
    inputs = [
        f"Question: {prompt} Context: {context}"
        for prompt, context in zip(examples["sql_prompt"], examples["sql_context"])
    ]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    # Tokenize targets (SQL queries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["sql"],
            max_length=128,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

    model_inputs["labels"] = labels["input_ids"]

    # Replace padding token id's of the labels by -100 so it's ignored by the loss
    model_inputs["labels"] = torch.where(
        model_inputs["labels"] == tokenizer.pad_token_id, -100, model_inputs["labels"]
    )

    return model_inputs


# Updated data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
    return_tensors="pt",
)

np.random.seed(42)
# Prepare datasets
print("Preparing training dataset...")
train_indices = random.sample(
    range(len(dataset["train"])), int(0.01 * len(dataset["train"]))
)
train_dataset = (
    dataset["train"]
    .select(train_indices)
    .map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
        desc="Processing training data",
    )
)


print("Preparing validation dataset...")
val_indices = random.sample(
    range(len(dataset["test"])), int(0.01 * len(dataset["test"]))
)
val_dataset = (
    dataset["test"]
    .select(val_indices)
    .map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["test"].column_names,
        desc="Processing validation data",
    )
)

# Debug dataset structure
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Training arguments with epochs and evaluation
training_args = TrainingArguments(
    output_dir="./flan-t5-large-sql-fullft",
    num_train_epochs=1,  # Train for 3 epochs
    eval_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",  # Save after each epoch
    learning_rate=1e-5,  # Lower learning rate for full fine-tuning
    per_device_train_batch_size=1,  # Smaller batch size for memory constraints
    per_device_eval_batch_size=1,
    # warmup_steps=50,
    logging_steps=5,
    save_total_limit=2,  # Keep last 3 checkpoints
    # load_best_model_at_end=True,
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,
    # dataloader_pin_memory=False,
    # remove_unused_columns=False, # Removed this line
    # Add evaluation metrics
    # eval_steps=None,  # Remove this when using epoch strategy,
    # Add these for speed
    # dataloader_num_workers=0,       # Disable multiprocessing
    report_to=None,
    # Memory optimization for full fine-tuning
    gradient_checkpointing=True,
    dataloader_num_workers=0,
    # Add gradient accumulation for effective larger batch size
    gradient_accumulation_steps=8,
    # Add weight decay for regularization
    weight_decay=0.01,
    # fp16=True  # Enable mixed precision training
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

Preparing training dataset...


Processing training data:   0%|          | 0/1000 [00:00<?, ? examples/s]/Users/pushking/Projects@Adobe/StudyMaterial/IITGenAIDocs/Code/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3950: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Processing training data: 100%|██████████| 1000/1000 [00:00<00:00, 2635.26 examples/s]


Preparing validation dataset...


Processing validation data: 100%|██████████| 58/58 [00:00<00:00, 1838.59 examples/s]

Training dataset size: 1000
Validation dataset size: 58


#### Fine tuning the model

In [10]:
# Start fine-tuning
print("Starting Full fine-tuning...")
trainer.train()

# Save the fully fine-tuned model
trainer.save_model("./flan-t5-large-sql-fullft-final")
print("Full fine-tuning completed and model saved!")
    

Starting Full fine-tuning...


/Users/pushking/Projects@Adobe/StudyMaterial/IITGenAIDocs/Code/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

#### Benchmarking finetuned model on Test Dataset

In [11]:
# read the saved model under project folder and evaluate the performance on the test set
    
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
print(device)

# Load the base model and tokenizer
fulltuned_model = T5ForConditionalGeneration.from_pretrained("./flan-t5-large-sql-fullft-final")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
fulltuned_model.to(device)

print("Model loaded successfully from checkpoint!")
    
# Run evaluation
results, generated_sqls, reference_sqls, test_dataset = evaluate_sql_generation(fulltuned_model, tokenizer, test_samples=10)

# Print results
print("\n" + "="*50)
print("EVALUATION RESULTS FOR T5 LARGE FINETUNED WITH LORA MODEL")
print("="*50)

metrics_summary = {}
for metric_name, scores in results.items():
        if scores:
            mean_score = np.mean(scores)
            std_score = np.std(scores)
            metrics_summary[metric_name] = {
                'mean': mean_score,
                'std': std_score
            }
            print(f"{metric_name.replace('_', ' ').title()}: {mean_score:.4f} ± {std_score:.4f}")    

# Print some examples
print("\n" + "="*50)
print("SAMPLE GENERATIONS")
print("="*50)

for i in range(min(10, len(generated_sqls))):
        print(f"\nExample {i+1}:")
        print(f"Actual: {test_dataset[i]['sql']}")
        print(f"SQL Prompt: {test_dataset[i]['sql_prompt']}")
        print(f"SQL Context: {test_dataset[i]['sql_context']}")
        print(f"Generated: {generated_sqls[i]}")
        print(f"BLEU: {results['bleu_scores'][i]:.4f}")
        print(f"ROUGE-L: {results['rouge_l_scores'][i]:.4f}")
        print(f"METEOR: {results['meteor_scores'][i]:.4f}")
        print(f"GLEU: {results['gleu_scores'][i]:.4f}")
        print(f"Flesch Reading Ease: {results['flesch_reading_ease_scores'][i]:.4f}")
        print(f"BERT: {results['bert_scores'][i]:.4f}")
        print(f"CoSIM: {results['cosim_scores'][i]:.4f}")
        print(f"Repetition Rate: {results['repetition_rates'][i]:.4f}")
        print(f"Novelty: {results['novelty_scores'][i]:.4f}")
        print(f"Diversity: {results['diversity_scores'][i]:.4f}")   
        print(f"Toxicity: {results['toxicity_scores'][i]:.4f}")   

cpu
Model loaded successfully from checkpoint!


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/pushking/Projects@Adobe/StudyMaterial/IITGenAIDocs/Code/venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Users/pushking/Projects@Adobe/StudyMaterial/IITGenAIDocs/Code/venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Processed 1/10 samples


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


EVALUATION RESULTS FOR T5 LARGE FINETUNED WITH LORA MODEL
Bleu Scores: 0.1132 ± 0.1208
Rouge 1 Scores: 0.4688 ± 0.2088
Rouge 2 Scores: 0.2393 ± 0.2645
Rouge L Scores: 0.3780 ± 0.2179
Meteor Scores: 0.3252 ± 0.2044
Gleu Scores: 0.4883 ± 0.1776
Flesch Reading Ease Scores: -7.0155 ± 47.8448
Bert Scores: 0.8910 ± 0.0373
Cosim Scores: 0.7334 ± 0.1178
Repetition Rates: 0.0311 ± 0.0399
Novelty Scores: 0.2574 ± 0.0332
Diversity Scores: 1.0000 ± 0.0000
Toxicity Scores: 0.0006 ± 0.0000

SAMPLE GENERATIONS

Example 1:
Actual: SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');
SQL Prompt: What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?
SQL Context: CREATE TABLE creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT); INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'Appl

In [ ]:
# This method takes an input text along with context (optional) and returns a SQL query.
# It evaluates the SQL query using the evaluate_sql_generation function and prints the results.


def evaluate_sample_sql_generation(model, tokenizer, sample_dataset):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Running model on device type {device}")

    # Run evaluation
    results, generated_sqls, reference_sqls, test_dataset = evaluate_sql_generation(
        model, tokenizer, test_samples=0, sample_dataset=sample_dataset
    )

    # Print results
    print("\n" + "=" * 50)
    print("EVALUATION RESULTS FOR T5 LARGE FINETUNED WITH LORA MODEL")
    print("=" * 50)

    metrics_summary = {}
    for metric_name, scores in results.items():
        if scores:
            mean_score = np.mean(scores)
            std_score = np.std(scores)
            metrics_summary[metric_name] = {"mean": mean_score, "std": std_score}
            print(
                f"{metric_name.replace('_', ' ').title()}: {mean_score:.4f} ± {std_score:.4f}"
            )

    # Print some examples
    print("\n" + "=" * 50)
    print("SAMPLE GENERATIONS")
    print("=" * 50)
    for i in range(min(10, len(generated_sqls))):
        print(f"\nExample {i+1}:")
        print(f"Actual: {test_dataset[i]['sql']}")
        print(f"SQL Prompt: {test_dataset[i]['sql_prompt']}")
        print(f"SQL Context: {test_dataset[i]['sql_context']}")
        print(f"Generated: {generated_sqls[i]}")
        print(f"BLEU: {results['bleu_scores'][i]:.4f}")
        print(f"ROUGE-L: {results['rouge_l_scores'][i]:.4f}")
        print(f"METEOR: {results['meteor_scores'][i]:.4f}")
        print(f"GLEU: {results['gleu_scores'][i]:.4f}")
        print(f"Flesch Reading Ease: {results['flesch_reading_ease_scores'][i]:.4f}")
        print(f"BERT: {results['bert_scores'][i]:.4f}")
        print(f"CoSIM: {results['cosim_scores'][i]:.4f}")
        print(f"Repetition Rate: {results['repetition_rates'][i]:.4f}")
        print(f"Novelty: {results['novelty_scores'][i]:.4f}")
        print(f"Diversity: {results['diversity_scores'][i]:.4f}")
        print(f"Toxicity: {results['toxicity_scores'][i]:.4f}")


# Load the base model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
fulltuned_model = T5ForConditionalGeneration.from_pretrained(
    "./flan-t5-large-sql-fullft-final"
)
fulltuned_model.to(device)
print("Model loaded successfully from checkpoint!")

input_text = "What is the total number of employees in the company?"
context = "CREATE TABLE employees (id INT, name VARCHAR(255), department VARCHAR(255), salary INT)"
# context = ""
reference_sql = "SELECT COUNT(*) AS total_employees FROM employees"

# convert to dataset format
sample_dataset = [
    {"sql_prompt": input_text, "sql_context": context, "sql": reference_sql}
]

evaluate_sample_sql_generation(fulltuned_model, tokenizer, sample_dataset)